In [1]:
import numpy as np
import random
import pandas as pd
import os 
import geopandas as gpd 
from msmla50 import MSMLA50
import gc
import utils
import torch
import cnn_utils
import pickle

torch.manual_seed(0)
np.random.seed(0)
torch.cuda.manual_seed(0)
random.seed(0)
torch.cuda.manual_seed_all(0)
torch.backends.cudnn.benchmark = False
torch.backends.cudnn.deterministic = True
torch.use_deterministic_algorithms(True, warn_only=True)
os.environ["CUBLAS_WORKSPACE_CONFIG"] = ":4096:8"
np.random.default_rng(seed=0)

Generator(PCG64) at 0x1CE95495FC0

In [2]:
# settings
patch_size = 32
stride = 10
gt_stride = 32
background_label = 0
batch_size = 128
offset_left = 'best'
offset_top = 'best'

# Berlin

In [3]:
# load splited reference data
splited_ref_data = gpd.read_file(r'ref_data\berlin_ref_splitS2S3S4.gpkg')

In [4]:
# load satellite image (10 m resolution)
image = r'imagery\berlin_20170519.tif'

In [5]:
# load trained S2 CNN models for each fold
cnn_models = [r's2_cnn_models\berlin_S2_fold0_epoch12.pth',
              r's2_cnn_models\berlin_S2_fold1_epoch16.pth',
              r's2_cnn_models\berlin_S2_fold2_epoch22.pth',
              r's2_cnn_models\berlin_S2_fold3_epoch39.pth',
              r's2_cnn_models\berlin_S2_fold4_epoch32.pth']

In [6]:
# define outputs
outputs = [r'outputs\s2\berlin_S2_fold0.tif',
          r'outputs\s2\berlin_S2_fold1.tif',
          r'outputs\s2\berlin_S2_fold2.tif',
          r'outputs\s2\berlin_S2_fold3.tif',
          r'outputs\s2\berlin_S2_fold4.tif']

## Prediction

In [7]:
# recording results
folds = [0, 1, 2, 3, 4]

# results[fold]
results = {
    fold: {} for fold in folds
}

In [8]:
feature_patches = cnn_utils.generate_feature_patches_loader(image_path = image,patch_size = patch_size,stride = stride,batch_size = batch_size,offset_left = offset_left,offset_top = offset_top)

Total patches loaded: 418465


In [9]:
for fold in folds:
    ## prepare training and test polygons
    test_polygons = splited_ref_data[splited_ref_data["fold"] == fold]
    train_polygons = splited_ref_data[splited_ref_data["fold"] != fold]

    train_polygons_raster = fr"berlin_train_f{fold}.tif"
    test_polygons_raster = fr"berlin_test_f{fold}.tif"

    # rasterize
    train_temp = train_polygons_raster.replace(".tif", "_temp.tif")
    test_temp = test_polygons_raster.replace(".tif", "_temp.tif")
    utils.rasterize_reference_polygons(train_polygons, image, train_temp)
    utils.rasterize_reference_polygons(test_polygons, image, test_temp)

    # train and test images matched to 10m image
    train_image_matched = utils.match_rasters(train_temp, image)
    test_image_matched = utils.match_rasters(test_temp, image)

    # save
    train_image_matched.rio.to_raster(train_polygons_raster, driver="GTiff", compress="LZW")
    test_image_matched.rio.to_raster(test_polygons_raster, driver="GTiff", compress="LZW")

    # cleaning
    train_image_matched.close()
    test_image_matched.close()
    train_image_matched = None
    test_image_matched = None
    gc.collect()
    if os.path.exists(train_temp):
        os.remove(train_temp)
    if os.path.exists(test_temp):
        os.remove(test_temp)

    ## load trained CNN model
    cnn_model = MSMLA50(input_channels=10, depth=[16,32,48], num_classes=len(train_polygons["gridcode"].unique()))
    cnn_model = cnn_model.cuda()
    trained_model = torch.load(cnn_models[fold])
    cnn_model.load_state_dict(trained_model['model_state'])

    ## get training patches
    train_patches = cnn_utils.generate_labeled_patches_loader(image_path = image,reference_path = train_polygons_raster,patch_size = patch_size,stride = gt_stride,batch_size = batch_size,offset_left = offset_left,offset_top = offset_top,background_label = background_label)

    ## remapping labels
    label_mapping = cnn_utils.compute_label_mapping(train_patches)
    train_patches = cnn_utils.label_remapping(train_patches, label_mapping)

    ## normalize patches
    mean, std = cnn_utils.get_normalization_parameters(train_patches)
    feature_patches_norm = cnn_utils.normalize_loader(feature_patches, mean, std)

    ## prediction
    print(f"Prediction started for fold {fold}...")
    cnn_model.eval()
    all_preds = list()
    with torch.no_grad():
        for features in feature_patches_norm:
            features = features.cuda()
            
            pred = cnn_model(features)
            pred = pred.cpu()
            
            _, predicted = torch.max(pred, 1)
            all_preds.append(predicted)
    y_pred = torch.cat(all_preds, dim=0)

    ## remapping labels back
    inverse_mapping = {new_label: old_label for old_label, new_label in label_mapping.items()}
    predicted_original_labels = np.array([inverse_mapping[label] for label in y_pred.numpy()])

    ## generate lcz map
    offset_left_calc, offset_top_calc = cnn_utils.calculate_optimal_offsets(image, patch_size, stride)
    utils.lcz_map(offset_left_calc, offset_top_calc, image, predicted_original_labels, outputs[fold])

    print(f"Fold {fold} predicted and saved to {outputs[fold]}.")

Total ground truth patches generated: 1956
Unique Labels: [ 2  4  5  6  8  9 11 12 13 14 16 17]
Counts: [116  34 202 317 119  69 395  81  85 361  29 148]
Original unique label values: [ 2  4  5  6  8  9 11 12 13 14 16 17], Counts: [116  34 202 317 119  69 395  81  85 361  29 148]
Remapped unique label values: [ 0  1  2  3  4  5  6  7  8  9 10 11], Counts: [116  34 202 317 119  69 395  81  85 361  29 148]
Prediction started for fold 0...
 saved to s2_outputs\berlin_S2_fold0.tif
Fold 0 predicted and saved to s2_outputs\berlin_S2_fold0.tif.
Total ground truth patches generated: 1830
Unique Labels: [ 2  4  5  6  8  9 11 12 13 14 16 17]
Counts: [102  41 195 320 124  53 383  76  72 320  26 118]
Original unique label values: [ 2  4  5  6  8  9 11 12 13 14 16 17], Counts: [102  41 195 320 124  53 383  76  72 320  26 118]
Remapped unique label values: [ 0  1  2  3  4  5  6  7  8  9 10 11], Counts: [102  41 195 320 124  53 383  76  72 320  26 118]
Prediction started for fold 1...
 saved to s2_ou

## Perpixel validation

In [10]:
# provide test polygons raster path
test_polygons_path = ['berlin_test_f0.tif','berlin_test_f1.tif','berlin_test_f2.tif','berlin_test_f3.tif','berlin_test_f4.tif']

In [11]:
# resample lcz map to 100m
resampled_outputs = []
for f in outputs:
    out = f.replace(".tif", "_100m.tif")
    utils.resample_lcz_map(f, out)
    resampled_outputs.append(out)
    # if os.path.exists(f):
    #     try:
    #         os.remove(f)
    #     except:
    #         pass

 saved to s2_outputs\berlin_S2_fold0_100m.tif
 saved to s2_outputs\berlin_S2_fold1_100m.tif
 saved to s2_outputs\berlin_S2_fold2_100m.tif
 saved to s2_outputs\berlin_S2_fold3_100m.tif
 saved to s2_outputs\berlin_S2_fold4_100m.tif


In [12]:
metrics, confusion_matrices = utils.perpixel_validation(resampled_outputs, test_polygons_path, splited_ref_data)

In [13]:
df_perpixel = pd.DataFrame(metrics)
df_perpixel = df_perpixel.set_index("Fold")
df_perpixel

,OA,wF1,wF1_Urban,wF1_Natural,F1_Class_1,F1_Class_2,F1_Class_3,F1_Class_4,F1_Class_5,F1_Class_6,...,F1_Class_8,F1_Class_9,F1_Class_10,F1_Class_11,F1_Class_12,F1_Class_13,F1_Class_14,F1_Class_15,F1_Class_16,F1_Class_17
Fold,,,,,,,,,,,,,,,,,,,,,
0,76.10,74.48,71.08,79.19,NaN,88.23,NaN,0.0,59.31,89.60,...,79.02,0.00,NaN,94.47,34.65,19.47,77.77,NaN,0.00,99.81
1,80.54,77.22,66.94,85.02,NaN,86.18,NaN,0.0,61.94,82.45,...,77.45,0.00,NaN,96.95,66.09,19.85,87.77,NaN,59.49,100.00
2,79.05,75.96,67.93,83.96,NaN,83.30,NaN,0.0,54.56,79.80,...,78.89,47.62,NaN,97.84,52.29,0.00,93.46,NaN,22.86,98.06
3,85.60,83.87,87.07,82.78,NaN,73.66,NaN,0.0,79.96,96.80,...,69.96,87.79,NaN,95.34,59.09,43.79,94.43,NaN,9.94,100.00
4,85.12,83.64,83.09,85.69,NaN,85.80,NaN,0.0,76.99,94.65,...,82.47,78.10,NaN,94.10,7.69,36.43,95.84,NaN,74.58,100.00


In [14]:
df_perpixel_mean = df_perpixel.mean().round(2)
df_perpixel_mean

OA             81.28
wF1            79.03
wF1_Urban      75.22
wF1_Natural    83.33
F1_Class_1       NaN
F1_Class_2     83.43
F1_Class_3       NaN
F1_Class_4      0.00
F1_Class_5     66.55
F1_Class_6     88.66
F1_Class_7       NaN
F1_Class_8     77.56
F1_Class_9     42.70
F1_Class_10      NaN
F1_Class_11    95.74
F1_Class_12    43.96
F1_Class_13    23.91
F1_Class_14    89.85
F1_Class_15      NaN
F1_Class_16    33.37
F1_Class_17    99.57
dtype: float64

In [15]:
# export all results to csv
df_perpixel.to_csv(r"results\s2\berlin_S2_results.csv")

In [16]:
# export confusion matrices
with open(r"results\s2\berlin_S2_confusion_matrices.pkl", "wb") as f:
    pickle.dump(confusion_matrices, f)

# Hong Kong

In [3]:
# load splited reference data
splited_ref_data = gpd.read_file(r'ref_data\hongkong_ref_splitS2S3S4.gpkg')

In [4]:
# load satellite image (10 m resolution)
image = r'imagery\hongkong_20180321.tif'

In [5]:
# load trained S2 CNN models for each fold
cnn_models = [r's2_cnn_models\hongkong_S2_fold0_epoch43.pth',
              r's2_cnn_models\hongkong_S2_fold1_epoch38.pth',
              r's2_cnn_models\hongkong_S2_fold2_epoch33.pth',
              r's2_cnn_models\hongkong_S2_fold3_epoch26.pth',
              r's2_cnn_models\hongkong_S2_fold4_epoch29.pth']

In [6]:
# define outputs
outputs = [r'outputs\s2\hongkong_S2_fold0.tif',
          r'outputs\s2\hongkong_S2_fold1.tif',
          r'outputs\s2\hongkong_S2_fold2.tif',
          r'outputs\s2\hongkong_S2_fold3.tif',
          r'outputs\s2\hongkong_S2_fold4.tif']

## Prediction

In [7]:
# recording results
folds = [0, 1, 2, 3, 4]

# results[fold]
results = {
    fold: {} for fold in folds
}

In [8]:
feature_patches = cnn_utils.generate_feature_patches_loader(image_path = image,patch_size = patch_size,stride = stride,batch_size = batch_size,offset_left = offset_left,offset_top = offset_top)

Total patches loaded: 214720


In [9]:
for fold in folds:
    ## prepare training and test polygons
    test_polygons = splited_ref_data[splited_ref_data["fold"] == fold]
    train_polygons = splited_ref_data[splited_ref_data["fold"] != fold]

    train_polygons_raster = fr"hongkong_train_f{fold}.tif"
    test_polygons_raster = fr"hongkong_test_f{fold}.tif"

    # rasterize
    train_temp = train_polygons_raster.replace(".tif", "_temp.tif")
    test_temp = test_polygons_raster.replace(".tif", "_temp.tif")
    utils.rasterize_reference_polygons(train_polygons, image, train_temp)
    utils.rasterize_reference_polygons(test_polygons, image, test_temp)

    # train and test images matched to 10m image
    train_image_matched = utils.match_rasters(train_temp, image)
    test_image_matched = utils.match_rasters(test_temp, image)

    # save
    train_image_matched.rio.to_raster(train_polygons_raster, driver="GTiff", compress="LZW")
    test_image_matched.rio.to_raster(test_polygons_raster, driver="GTiff", compress="LZW")

    # cleaning
    train_image_matched.close()
    test_image_matched.close()
    train_image_matched = None
    test_image_matched = None
    gc.collect()
    if os.path.exists(train_temp):
        os.remove(train_temp)
    if os.path.exists(test_temp):
        os.remove(test_temp)

    ## load trained CNN model
    cnn_model = MSMLA50(input_channels=10, depth=[16,32,48], num_classes=len(train_polygons["gridcode"].unique()))
    cnn_model = cnn_model.cuda()
    trained_model = torch.load(cnn_models[fold])
    cnn_model.load_state_dict(trained_model['model_state'])

    ## get training patches
    train_patches = cnn_utils.generate_labeled_patches_loader(image_path = image,reference_path = train_polygons_raster,patch_size = patch_size,stride = gt_stride,batch_size = batch_size,offset_left = offset_left,offset_top = offset_top,background_label = background_label)

    ## remapping labels
    label_mapping = cnn_utils.compute_label_mapping(train_patches)
    train_patches = cnn_utils.label_remapping(train_patches, label_mapping)

    ## normalize patches
    mean, std = cnn_utils.get_normalization_parameters(train_patches)
    feature_patches_norm = cnn_utils.normalize_loader(feature_patches, mean, std)

    ## prediction
    print(f"Prediction started for fold {fold}...")
    cnn_model.eval()
    all_preds = list()
    with torch.no_grad():
        for features in feature_patches_norm:
            features = features.cuda()
            
            pred = cnn_model(features)
            pred = pred.cpu()
            
            _, predicted = torch.max(pred, 1)
            all_preds.append(predicted)
    y_pred = torch.cat(all_preds, dim=0)

    ## remapping labels back
    inverse_mapping = {new_label: old_label for old_label, new_label in label_mapping.items()}
    predicted_original_labels = np.array([inverse_mapping[label] for label in y_pred.numpy()])

    ## generate lcz map
    offset_left_calc, offset_top_calc = cnn_utils.calculate_optimal_offsets(image, patch_size, stride)
    utils.lcz_map(offset_left_calc, offset_top_calc, image, predicted_original_labels, outputs[fold])

    print(f"Fold {fold} predicted and saved to {outputs[fold]}.")

Total ground truth patches generated: 706
Unique Labels: [ 1  2  3  4  5  6  8 10 11 12 13 14 17]
Counts: [ 45  12  27  55   8   8  10  22 122  46  56  75 220]
Original unique label values: [ 1  2  3  4  5  6  8 10 11 12 13 14 17], Counts: [ 45  12  27  55   8   8  10  22 122  46  56  75 220]
Remapped unique label values: [ 0  1  2  3  4  5  6  7  8  9 10 11 12], Counts: [ 45  12  27  55   8   8  10  22 122  46  56  75 220]
Prediction started for fold 0...
 saved to s2_outputs\hongkong_S2_fold0.tif
Fold 0 predicted and saved to s2_outputs\hongkong_S2_fold0.tif.
Total ground truth patches generated: 658
Unique Labels: [ 1  2  3  4  5  6  8 10 11 12 13 14 17]
Counts: [ 44  15  24  45   7   8  11  18 129  40  53  65 199]
Original unique label values: [ 1  2  3  4  5  6  8 10 11 12 13 14 17], Counts: [ 44  15  24  45   7   8  11  18 129  40  53  65 199]
Remapped unique label values: [ 0  1  2  3  4  5  6  7  8  9 10 11 12], Counts: [ 44  15  24  45   7   8  11  18 129  40  53  65 199]
Pred

## Perpixel validation

In [10]:
# provide test polygons raster path
test_polygons_path = ['hongkong_test_f0.tif','hongkong_test_f1.tif','hongkong_test_f2.tif','hongkong_test_f3.tif','hongkong_test_f4.tif']

In [11]:
# resample lcz map to 100m
resampled_outputs = []
for f in outputs:
    out = f.replace(".tif", "_100m.tif")
    utils.resample_lcz_map(f, out)
    resampled_outputs.append(out)
    # if os.path.exists(f):
    #     try:
    #         os.remove(f)
    #     except:
    #         pass

 saved to s2_outputs\hongkong_S2_fold0_100m.tif
 saved to s2_outputs\hongkong_S2_fold1_100m.tif
 saved to s2_outputs\hongkong_S2_fold2_100m.tif
 saved to s2_outputs\hongkong_S2_fold3_100m.tif
 saved to s2_outputs\hongkong_S2_fold4_100m.tif


In [12]:
metrics, confusion_matrices = utils.perpixel_validation(resampled_outputs, test_polygons_path, splited_ref_data)

In [13]:
df_perpixel = pd.DataFrame(metrics)
df_perpixel = df_perpixel.set_index("Fold")
df_perpixel

,OA,wF1,wF1_Urban,wF1_Natural,F1_Class_1,F1_Class_2,F1_Class_3,F1_Class_4,F1_Class_5,F1_Class_6,...,F1_Class_8,F1_Class_9,F1_Class_10,F1_Class_11,F1_Class_12,F1_Class_13,F1_Class_14,F1_Class_15,F1_Class_16,F1_Class_17
Fold,,,,,,,,,,,,,,,,,,,,,
0,73.27,70.59,45.02,81.12,47.35,0.00,59.26,64.29,0.00,0.00,...,37.50,NaN,74.00,93.05,19.78,46.98,60.25,NaN,NaN,99.44
1,72.79,69.34,70.80,69.32,76.88,0.00,89.71,88.42,46.15,33.33,...,0.00,NaN,30.23,89.19,51.16,48.85,0.00,NaN,NaN,98.43
2,71.46,69.07,49.81,76.68,75.31,37.50,53.66,67.65,8.33,16.33,...,0.00,NaN,59.57,90.48,26.23,10.99,64.62,NaN,NaN,99.40
3,70.73,69.31,56.05,77.93,51.80,27.85,67.52,73.65,0.00,0.00,...,8.70,NaN,0.00,96.46,55.24,0.00,74.70,NaN,NaN,87.09
4,77.12,74.02,61.38,80.29,82.17,0.00,73.51,74.05,0.00,0.00,...,29.63,NaN,85.71,91.79,34.69,67.89,56.80,NaN,NaN,99.87


In [14]:
df_perpixel_mean = df_perpixel.mean().round(2)
df_perpixel_mean

OA             73.07
wF1            70.47
wF1_Urban      56.61
wF1_Natural    77.07
F1_Class_1     66.70
F1_Class_2     13.07
F1_Class_3     68.73
F1_Class_4     73.61
F1_Class_5     10.90
F1_Class_6      9.93
F1_Class_7       NaN
F1_Class_8     15.17
F1_Class_9       NaN
F1_Class_10    49.90
F1_Class_11    92.19
F1_Class_12    37.42
F1_Class_13    34.94
F1_Class_14    51.27
F1_Class_15      NaN
F1_Class_16      NaN
F1_Class_17    96.85
dtype: float64

In [15]:
# export all results to csv
df_perpixel.to_csv(r"results\s2\hongkong_S2_results.csv")

In [19]:
# export confusion matrices
with open(r"results\s2\hongkong_S2_confusion_matrices.pkl", "wb") as f:
    pickle.dump(confusion_matrices, f)

# Paris

In [17]:
# load splited reference data
splited_ref_data = gpd.read_file(r'ref_data\paris_ref_splitS2S3S4.gpkg')

In [18]:
# load satellite image (10 m resolution)
image = r'imagery\paris_20170526.tif'

In [19]:
# load trained S2 CNN models for each fold
cnn_models = [r's2_cnn_models\paris_S2_fold0_epoch64.pth',
              r's2_cnn_models\paris_S2_fold1_epoch88.pth',
              r's2_cnn_models\paris_S2_fold2_epoch75.pth',
              r's2_cnn_models\paris_S2_fold3_epoch76.pth',
              r's2_cnn_models\paris_S2_fold4_epoch88.pth']

In [20]:
# define outputs
outputs = [r'outputs\s2\paris_S2_fold0.tif',
          r'outputs\s2\paris_S2_fold1.tif',
          r'outputs\s2\paris_S2_fold2.tif',
          r'outputs\s2\paris_S2_fold3.tif',
          r'outputs\s2\paris_S2_fold4.tif']

## Prediction

In [21]:
# recording results
folds = [0, 1, 2, 3, 4]

# results[fold]
results = {
    fold: {} for fold in folds
}

In [22]:
feature_patches = cnn_utils.generate_feature_patches_loader(image_path = image,patch_size = patch_size,stride = stride,batch_size = batch_size,offset_left = offset_left,offset_top = offset_top)

Total patches loaded: 976752


In [23]:
for fold in folds:
    ## prepare training and test polygons
    test_polygons = splited_ref_data[splited_ref_data["fold"] == fold]
    train_polygons = splited_ref_data[splited_ref_data["fold"] != fold]

    train_polygons_raster = fr"paris_train_f{fold}.tif"
    test_polygons_raster = fr"paris_test_f{fold}.tif"

    # rasterize
    train_temp = train_polygons_raster.replace(".tif", "_temp.tif")
    test_temp = test_polygons_raster.replace(".tif", "_temp.tif")
    utils.rasterize_reference_polygons(train_polygons, image, train_temp)
    utils.rasterize_reference_polygons(test_polygons, image, test_temp)

    # train and test images matched to 10m image
    train_image_matched = utils.match_rasters(train_temp, image)
    test_image_matched = utils.match_rasters(test_temp, image)

    # save
    train_image_matched.rio.to_raster(train_polygons_raster, driver="GTiff", compress="LZW")
    test_image_matched.rio.to_raster(test_polygons_raster, driver="GTiff", compress="LZW")

    # cleaning
    train_image_matched.close()
    test_image_matched.close()
    train_image_matched = None
    test_image_matched = None
    gc.collect()
    if os.path.exists(train_temp):
        os.remove(train_temp)
    if os.path.exists(test_temp):
        os.remove(test_temp)

    ## load trained CNN model
    cnn_model = MSMLA50(input_channels=10, depth=[16,32,48], num_classes=len(train_polygons["gridcode"].unique()))
    cnn_model = cnn_model.cuda()
    trained_model = torch.load(cnn_models[fold])
    cnn_model.load_state_dict(trained_model['model_state'])

    ## get training patches
    train_patches = cnn_utils.generate_labeled_patches_loader(image_path = image,reference_path = train_polygons_raster,patch_size = patch_size,stride = gt_stride,batch_size = batch_size,offset_left = offset_left,offset_top = offset_top,background_label = background_label)

    ## remapping labels
    label_mapping = cnn_utils.compute_label_mapping(train_patches)
    train_patches = cnn_utils.label_remapping(train_patches, label_mapping)

    ## normalize patches
    mean, std = cnn_utils.get_normalization_parameters(train_patches)
    feature_patches_norm = cnn_utils.normalize_loader(feature_patches, mean, std)

    ## prediction
    print(f"Prediction started for fold {fold}...")
    cnn_model.eval()
    all_preds = list()
    with torch.no_grad():
        for features in feature_patches_norm:
            features = features.cuda()
            
            pred = cnn_model(features)
            pred = pred.cpu()
            
            _, predicted = torch.max(pred, 1)
            all_preds.append(predicted)
    y_pred = torch.cat(all_preds, dim=0)

    ## remapping labels back
    inverse_mapping = {new_label: old_label for old_label, new_label in label_mapping.items()}
    predicted_original_labels = np.array([inverse_mapping[label] for label in y_pred.numpy()])

    ## generate lcz map
    offset_left_calc, offset_top_calc = cnn_utils.calculate_optimal_offsets(image, patch_size, stride)
    utils.lcz_map(offset_left_calc, offset_top_calc, image, predicted_original_labels, outputs[fold])

    print(f"Fold {fold} predicted and saved to {outputs[fold]}.")

Total ground truth patches generated: 1565
Unique Labels: [ 1  2  4  5  6  8  9 11 12 14 15 17]
Counts: [  4 240  27  31 187  66   2 357  31 583  17  20]
Original unique label values: [ 1  2  4  5  6  8  9 11 12 14 15 17], Counts: [  4 240  27  31 187  66   2 357  31 583  17  20]
Remapped unique label values: [ 0  1  2  3  4  5  6  7  8  9 10 11], Counts: [  4 240  27  31 187  66   2 357  31 583  17  20]
Prediction started for fold 0...
 saved to s2_outputs\paris_S2_fold0.tif
Fold 0 predicted and saved to s2_outputs\paris_S2_fold0.tif.
Total ground truth patches generated: 1391
Unique Labels: [ 1  2  4  5  6  8  9 11 12 14 15 17]
Counts: [  2  73  27  35 181  66   3 345  32 593  17  17]
Original unique label values: [ 1  2  4  5  6  8  9 11 12 14 15 17], Counts: [  2  73  27  35 181  66   3 345  32 593  17  17]
Remapped unique label values: [ 0  1  2  3  4  5  6  7  8  9 10 11], Counts: [  2  73  27  35 181  66   3 345  32 593  17  17]
Prediction started for fold 1...
 saved to s2_outp

## Perpixel validation

In [24]:
# provide test polygons raster path
test_polygons_path = ['paris_test_f0.tif','paris_test_f1.tif','paris_test_f2.tif','paris_test_f3.tif','paris_test_f4.tif']

In [25]:
# resample lcz map to 100m
resampled_outputs = []
for f in outputs:
    out = f.replace(".tif", "_100m.tif")
    utils.resample_lcz_map(f, out)
    resampled_outputs.append(out)
    # if os.path.exists(f):
    #     try:
    #         os.remove(f)
    #     except:
    #         pass

 saved to s2_outputs\paris_S2_fold0_100m.tif
 saved to s2_outputs\paris_S2_fold1_100m.tif
 saved to s2_outputs\paris_S2_fold2_100m.tif
 saved to s2_outputs\paris_S2_fold3_100m.tif
 saved to s2_outputs\paris_S2_fold4_100m.tif


In [26]:
metrics, confusion_matrices = utils.perpixel_validation(resampled_outputs, test_polygons_path, splited_ref_data)

In [27]:
df_perpixel = pd.DataFrame(metrics)
df_perpixel = df_perpixel.set_index("Fold")
df_perpixel

,OA,wF1,wF1_Urban,wF1_Natural,F1_Class_1,F1_Class_2,F1_Class_3,F1_Class_4,F1_Class_5,F1_Class_6,...,F1_Class_8,F1_Class_9,F1_Class_10,F1_Class_11,F1_Class_12,F1_Class_13,F1_Class_14,F1_Class_15,F1_Class_16,F1_Class_17
Fold,,,,,,,,,,,,,,,,,,,,,
0,88.47,87.69,75.29,93.67,0.00,87.77,NaN,23.65,40.61,91.09,...,66.26,0.00,NaN,91.89,36.57,NaN,98.53,0.00,NaN,88.00
1,95.51,95.24,93.49,97.80,42.31,99.52,NaN,60.82,43.54,90.49,...,77.84,53.57,NaN,98.19,85.71,NaN,98.64,16.00,NaN,98.51
2,88.60,89.21,83.17,93.89,NaN,89.43,NaN,42.03,43.43,88.41,...,75.11,NaN,NaN,97.09,61.86,NaN,97.29,19.44,NaN,90.91
3,91.18,90.54,85.16,93.76,NaN,88.81,NaN,51.52,59.41,92.05,...,71.43,NaN,NaN,97.72,0.00,NaN,98.98,0.00,NaN,83.72
4,95.62,95.73,90.93,98.16,NaN,95.83,NaN,54.00,58.93,98.12,...,89.17,NaN,NaN,98.98,76.51,NaN,99.60,53.33,NaN,93.27


In [28]:
df_perpixel_mean = df_perpixel.mean().round(2)
df_perpixel_mean

OA             91.88
wF1            91.68
wF1_Urban      85.61
wF1_Natural    95.46
F1_Class_1     21.16
F1_Class_2     92.27
F1_Class_3       NaN
F1_Class_4     46.40
F1_Class_5     49.18
F1_Class_6     92.03
F1_Class_7       NaN
F1_Class_8     75.96
F1_Class_9     26.78
F1_Class_10      NaN
F1_Class_11    96.77
F1_Class_12    52.13
F1_Class_13      NaN
F1_Class_14    98.61
F1_Class_15    17.75
F1_Class_16      NaN
F1_Class_17    90.88
dtype: float64

In [29]:
# export all results to csv
df_perpixel.to_csv(r"results\s2\paris_S2_results.csv")

In [30]:
# export confusion matrices
with open(r"results\s2\paris_S2_confusion_matrices.pkl", "wb") as f:
    pickle.dump(confusion_matrices, f)

# Rome

In [3]:
# load splited reference data
splited_ref_data = gpd.read_file(r'ref_data\rome_ref_splitS2S3S4.gpkg')

In [4]:
# load satellite image (10 m resolution)
image = r'imagery\rome_20170620.tif'

In [5]:
# load trained S2 CNN models for each fold
cnn_models = [r's2_cnn_models\rome_S2_fold0_epoch17.pth',
              r's2_cnn_models\rome_S2_fold1_epoch10.pth',
              r's2_cnn_models\rome_S2_fold2_epoch90.pth',
              r's2_cnn_models\rome_S2_fold3_epoch76.pth',
              r's2_cnn_models\rome_S2_fold4_epoch20.pth']

In [6]:
# define outputs
outputs = [r'outputs\s2\rome_S2_fold0.tif',
          r'outputs\s2\rome_S2_fold1.tif',
          r'outputs\s2\rome_S2_fold2.tif',
          r'outputs\s2\rome_S2_fold3.tif',
          r'outputs\s2\rome_S2_fold4.tif']

## Prediction

In [7]:
# recording results
folds = [0, 1, 2, 3, 4]

# results[fold]
results = {
    fold: {} for fold in folds
}

In [8]:
feature_patches = cnn_utils.generate_feature_patches_loader(image_path = image,patch_size = patch_size,stride = stride,batch_size = batch_size,offset_left = offset_left,offset_top = offset_top)

Total patches loaded: 181196


In [9]:
for fold in folds:
    ## prepare training and test polygons
    test_polygons = splited_ref_data[splited_ref_data["fold"] == fold]
    train_polygons = splited_ref_data[splited_ref_data["fold"] != fold]

    train_polygons_raster = fr"rome_train_f{fold}.tif"
    test_polygons_raster = fr"rome_test_f{fold}.tif"

    # rasterize
    train_temp = train_polygons_raster.replace(".tif", "_temp.tif")
    test_temp = test_polygons_raster.replace(".tif", "_temp.tif")
    utils.rasterize_reference_polygons(train_polygons, image, train_temp)
    utils.rasterize_reference_polygons(test_polygons, image, test_temp)

    # train and test images matched to 10m image
    train_image_matched = utils.match_rasters(train_temp, image)
    test_image_matched = utils.match_rasters(test_temp, image)

    # save
    train_image_matched.rio.to_raster(train_polygons_raster, driver="GTiff", compress="LZW")
    test_image_matched.rio.to_raster(test_polygons_raster, driver="GTiff", compress="LZW")

    # cleaning
    train_image_matched.close()
    test_image_matched.close()
    train_image_matched = None
    test_image_matched = None
    gc.collect()
    if os.path.exists(train_temp):
        os.remove(train_temp)
    if os.path.exists(test_temp):
        os.remove(test_temp)

    ## load trained CNN model
    cnn_model = MSMLA50(input_channels=10, depth=[16,32,48], num_classes=len(train_polygons["gridcode"].unique()))
    cnn_model = cnn_model.cuda()
    trained_model = torch.load(cnn_models[fold])
    cnn_model.load_state_dict(trained_model['model_state'])

    ## get training patches
    train_patches = cnn_utils.generate_labeled_patches_loader(image_path = image,reference_path = train_polygons_raster,patch_size = patch_size,stride = gt_stride,batch_size = batch_size,offset_left = offset_left,offset_top = offset_top,background_label = background_label)

    ## remapping labels
    label_mapping = cnn_utils.compute_label_mapping(train_patches)
    train_patches = cnn_utils.label_remapping(train_patches, label_mapping)

    ## normalize patches
    mean, std = cnn_utils.get_normalization_parameters(train_patches)
    feature_patches_norm = cnn_utils.normalize_loader(feature_patches, mean, std)

    ## prediction
    print(f"Prediction started for fold {fold}...")
    cnn_model.eval()
    all_preds = list()
    with torch.no_grad():
        for features in feature_patches_norm:
            features = features.cuda()
            
            pred = cnn_model(features)
            pred = pred.cpu()
            
            _, predicted = torch.max(pred, 1)
            all_preds.append(predicted)
    y_pred = torch.cat(all_preds, dim=0)

    ## remapping labels back
    inverse_mapping = {new_label: old_label for old_label, new_label in label_mapping.items()}
    predicted_original_labels = np.array([inverse_mapping[label] for label in y_pred.numpy()])

    ## generate lcz map
    offset_left_calc, offset_top_calc = cnn_utils.calculate_optimal_offsets(image, patch_size, stride)
    utils.lcz_map(offset_left_calc, offset_top_calc, image, predicted_original_labels, outputs[fold])

    print(f"Fold {fold} predicted and saved to {outputs[fold]}.")

Total ground truth patches generated: 485
Unique Labels: [ 2  3  5  6  8 10 11 12 14 17]
Counts: [125   3 111  40  36   3  24  42  83  18]
Original unique label values: [ 2  3  5  6  8 10 11 12 14 17], Counts: [125   3 111  40  36   3  24  42  83  18]
Remapped unique label values: [0 1 2 3 4 5 6 7 8 9], Counts: [125   3 111  40  36   3  24  42  83  18]
Prediction started for fold 0...
 saved to s2_outputs\rome_S2_fold0.tif
Fold 0 predicted and saved to s2_outputs\rome_S2_fold0.tif.
Total ground truth patches generated: 490
Unique Labels: [ 2  3  5  6  8 10 11 12 14 17]
Counts: [123   8 114  37  33   2  22  39  66  46]
Original unique label values: [ 2  3  5  6  8 10 11 12 14 17], Counts: [123   8 114  37  33   2  22  39  66  46]
Remapped unique label values: [0 1 2 3 4 5 6 7 8 9], Counts: [123   8 114  37  33   2  22  39  66  46]
Prediction started for fold 1...
 saved to s2_outputs\rome_S2_fold1.tif
Fold 1 predicted and saved to s2_outputs\rome_S2_fold1.tif.
Total ground truth patches

## Perpixel validation

In [10]:
# provide test polygons raster path
test_polygons_path = ['rome_test_f0.tif','rome_test_f1.tif','rome_test_f2.tif','rome_test_f3.tif','rome_test_f4.tif']

In [11]:
# resample lcz map to 100m
resampled_outputs = []
for f in outputs:
    out = f.replace(".tif", "_100m.tif")
    utils.resample_lcz_map(f, out)
    resampled_outputs.append(out)
    # if os.path.exists(f):
    #     try:
    #         os.remove(f)
    #     except:
    #         pass

 saved to s2_outputs\rome_S2_fold0_100m.tif
 saved to s2_outputs\rome_S2_fold1_100m.tif
 saved to s2_outputs\rome_S2_fold2_100m.tif
 saved to s2_outputs\rome_S2_fold3_100m.tif
 saved to s2_outputs\rome_S2_fold4_100m.tif


In [12]:
metrics, confusion_matrices = utils.perpixel_validation(resampled_outputs, test_polygons_path, splited_ref_data)

In [13]:
df_perpixel = pd.DataFrame(metrics)
df_perpixel = df_perpixel.set_index("Fold")
df_perpixel

,OA,wF1,wF1_Urban,wF1_Natural,F1_Class_1,F1_Class_2,F1_Class_3,F1_Class_4,F1_Class_5,F1_Class_6,...,F1_Class_8,F1_Class_9,F1_Class_10,F1_Class_11,F1_Class_12,F1_Class_13,F1_Class_14,F1_Class_15,F1_Class_16,F1_Class_17
Fold,,,,,,,,,,,,,,,,,,,,,
0,68.84,64.31,54.55,86.65,NaN,69.34,0.0,NaN,60.13,0.00,...,55.93,NaN,90.48,95.12,16.33,NaN,97.61,NaN,NaN,100.0
1,72.16,68.00,60.72,84.19,NaN,74.96,0.0,NaN,68.43,0.00,...,89.74,NaN,0.00,96.20,65.00,NaN,83.66,NaN,NaN,100.0
2,82.65,82.88,80.22,91.92,NaN,93.29,NaN,NaN,67.47,64.57,...,83.02,NaN,NaN,94.64,82.82,NaN,76.68,NaN,NaN,100.0
3,75.72,72.58,67.20,91.38,NaN,84.37,NaN,NaN,62.09,0.00,...,83.91,NaN,NaN,45.83,78.63,NaN,98.41,NaN,NaN,NaN
4,88.14,88.44,87.09,91.28,NaN,89.56,NaN,NaN,84.48,85.28,...,93.06,NaN,NaN,91.17,25.00,NaN,95.61,NaN,NaN,NaN


In [14]:
df_perpixel_mean = df_perpixel.mean().round(2)
df_perpixel_mean

OA              77.50
wF1             75.24
wF1_Urban       69.96
wF1_Natural     89.08
F1_Class_1        NaN
F1_Class_2      82.30
F1_Class_3       0.00
F1_Class_4        NaN
F1_Class_5      68.52
F1_Class_6      29.97
F1_Class_7        NaN
F1_Class_8      81.13
F1_Class_9        NaN
F1_Class_10     45.24
F1_Class_11     84.59
F1_Class_12     53.56
F1_Class_13       NaN
F1_Class_14     90.39
F1_Class_15       NaN
F1_Class_16       NaN
F1_Class_17    100.00
dtype: float64

In [15]:
# export all results to csv
df_perpixel.to_csv(r"results\s2\rome_S2_results.csv")

In [16]:
# export confusion matrices
with open(r"results\s2\rome_S2_confusion_matrices.pkl", "wb") as f:
    pickle.dump(confusion_matrices, f)

# Sao Paulo

In [31]:
# load splited reference data
splited_ref_data = gpd.read_file(r'ref_data\saopaulo_ref_splitS2S3S4.gpkg')

In [32]:
# load satellite image (10 m resolution)
image = r'imagery\sao_paulo_20170726.tif'

In [33]:
cnn_models = [r's2_cnn_models\saopaulo_S2_fold0_epoch67.pth',
              r's2_cnn_models\saopaulo_S2_fold1_epoch84.pth',
              r's2_cnn_models\saopaulo_S2_fold2_epoch67.pth',
              r's2_cnn_models\saopaulo_S2_fold3_epoch78.pth',
              r's2_cnn_models\saopaulo_S2_fold4_epoch37.pth']

In [34]:
# define outputs
outputs = [r'outputs\s2\saopaulo_S2_fold0.tif',
          r'outputs\s2\saopaulo_S2_fold1.tif',
          r'outputs\s2\saopaulo_S2_fold2.tif',
          r'outputs\s2\saopaulo_S2_fold3.tif',
          r'outputs\s2\saopaulo_S2_fold4.tif']

## Prediction

In [35]:
# recording results
folds = [0, 1, 2, 3, 4]

# results[fold]
results = {
    fold: {} for fold in folds
}

In [36]:
feature_patches = cnn_utils.generate_feature_patches_loader(image_path = image,patch_size = patch_size,stride = stride,batch_size = batch_size,offset_left = offset_left,offset_top = offset_top)

Total patches loaded: 508599


In [37]:
for fold in folds:
    ## prepare training and test polygons
    test_polygons = splited_ref_data[splited_ref_data["fold"] == fold]
    train_polygons = splited_ref_data[splited_ref_data["fold"] != fold]

    train_polygons_raster = fr"saopaulo_train_f{fold}.tif"
    test_polygons_raster = fr"saopaulo_test_f{fold}.tif"

    # rasterize
    train_temp = train_polygons_raster.replace(".tif", "_temp.tif")
    test_temp = test_polygons_raster.replace(".tif", "_temp.tif")
    utils.rasterize_reference_polygons(train_polygons, image, train_temp)
    utils.rasterize_reference_polygons(test_polygons, image, test_temp)

    # train and test images matched to 10m image
    train_image_matched = utils.match_rasters(train_temp, image)
    test_image_matched = utils.match_rasters(test_temp, image)

    # save
    train_image_matched.rio.to_raster(train_polygons_raster, driver="GTiff", compress="LZW")
    test_image_matched.rio.to_raster(test_polygons_raster, driver="GTiff", compress="LZW")

    # cleaning
    train_image_matched.close()
    test_image_matched.close()
    train_image_matched = None
    test_image_matched = None
    gc.collect()
    if os.path.exists(train_temp):
        os.remove(train_temp)
    if os.path.exists(test_temp):
        os.remove(test_temp)

    ## load trained CNN model
    cnn_model = MSMLA50(input_channels=10, depth=[16,32,48], num_classes=len(train_polygons["gridcode"].unique()))
    cnn_model = cnn_model.cuda()
    trained_model = torch.load(cnn_models[fold])
    cnn_model.load_state_dict(trained_model['model_state'])

    ## get training patches
    train_patches = cnn_utils.generate_labeled_patches_loader(image_path = image,reference_path = train_polygons_raster,patch_size = patch_size,stride = gt_stride,batch_size = batch_size,offset_left = offset_left,offset_top = offset_top,background_label = background_label)

    ## remapping labels
    label_mapping = cnn_utils.compute_label_mapping(train_patches)
    train_patches = cnn_utils.label_remapping(train_patches, label_mapping)

    ## normalize patches
    mean, std = cnn_utils.get_normalization_parameters(train_patches)
    feature_patches_norm = cnn_utils.normalize_loader(feature_patches, mean, std)

    ## prediction
    print(f"Prediction started for fold {fold}...")
    cnn_model.eval()
    all_preds = list()
    with torch.no_grad():
        for features in feature_patches_norm:
            features = features.cuda()
            
            pred = cnn_model(features)
            pred = pred.cpu()
            
            _, predicted = torch.max(pred, 1)
            all_preds.append(predicted)
    y_pred = torch.cat(all_preds, dim=0)

    ## remapping labels back
    inverse_mapping = {new_label: old_label for old_label, new_label in label_mapping.items()}
    predicted_original_labels = np.array([inverse_mapping[label] for label in y_pred.numpy()])

    ## generate lcz map
    offset_left_calc, offset_top_calc = cnn_utils.calculate_optimal_offsets(image, patch_size, stride)
    utils.lcz_map(offset_left_calc, offset_top_calc, image, predicted_original_labels, outputs[fold])

    print(f"Fold {fold} predicted and saved to {outputs[fold]}.")

Total ground truth patches generated: 1787
Unique Labels: [ 1  2  3  4  5  6  8  9 10 11 12 14 15 16 17]
Counts: [ 71  12 422  43  19 135 148  17   2 588  23  31   9  10 257]
Original unique label values: [ 1  2  3  4  5  6  8  9 10 11 12 14 15 16 17], Counts: [ 71  12 422  43  19 135 148  17   2 588  23  31   9  10 257]
Remapped unique label values: [ 0  1  2  3  4  5  6  7  8  9 10 11 12 13 14], Counts: [ 71  12 422  43  19 135 148  17   2 588  23  31   9  10 257]
Prediction started for fold 0...
 saved to s2_outputs\saopaulo_S2_fold0.tif
Fold 0 predicted and saved to s2_outputs\saopaulo_S2_fold0.tif.
Total ground truth patches generated: 1799
Unique Labels: [ 1  2  3  4  5  6  8  9 10 11 12 14 15 16 17]
Counts: [ 70  11 387  39  21 140 147  30  13 587  17  32   9  12 284]
Original unique label values: [ 1  2  3  4  5  6  8  9 10 11 12 14 15 16 17], Counts: [ 70  11 387  39  21 140 147  30  13 587  17  32   9  12 284]
Remapped unique label values: [ 0  1  2  3  4  5  6  7  8  9 10 11

## Perpixel validation

In [38]:
# provide test polygons raster path
test_polygons_path = ['saopaulo_test_f0.tif','saopaulo_test_f1.tif','saopaulo_test_f2.tif','saopaulo_test_f3.tif','saopaulo_test_f4.tif']

In [39]:
# resample lcz map to 100m
resampled_outputs = []
for f in outputs:
    out = f.replace(".tif", "_100m.tif")
    utils.resample_lcz_map(f, out)
    resampled_outputs.append(out)
    # if os.path.exists(f):
    #     try:
    #         os.remove(f)
    #     except:
    #         pass

 saved to s2_outputs\saopaulo_S2_fold0_100m.tif
 saved to s2_outputs\saopaulo_S2_fold1_100m.tif
 saved to s2_outputs\saopaulo_S2_fold2_100m.tif
 saved to s2_outputs\saopaulo_S2_fold3_100m.tif
 saved to s2_outputs\saopaulo_S2_fold4_100m.tif


In [40]:
metrics, confusion_matrices = utils.perpixel_validation(resampled_outputs, test_polygons_path, splited_ref_data)

In [41]:
df_perpixel = pd.DataFrame(metrics)
df_perpixel = df_perpixel.set_index("Fold")
df_perpixel

,OA,wF1,wF1_Urban,wF1_Natural,F1_Class_1,F1_Class_2,F1_Class_3,F1_Class_4,F1_Class_5,F1_Class_6,...,F1_Class_8,F1_Class_9,F1_Class_10,F1_Class_11,F1_Class_12,F1_Class_13,F1_Class_14,F1_Class_15,F1_Class_16,F1_Class_17
Fold,,,,,,,,,,,,,,,,,,,,,
0,83.86,82.90,80.15,91.41,92.24,0.00,94.70,55.88,17.44,75.32,...,85.21,68.95,0.00,88.18,43.30,NaN,41.03,0.0,19.72,99.94
1,89.42,89.28,88.67,92.38,89.68,37.74,97.71,57.30,0.00,84.38,...,87.27,41.18,19.05,98.07,73.68,NaN,55.64,0.0,15.00,99.65
2,88.22,88.31,82.17,92.62,85.64,0.00,92.57,57.73,0.00,80.95,...,90.45,62.28,NaN,95.06,58.12,NaN,53.73,0.0,20.36,92.97
3,86.46,86.25,79.76,94.84,79.08,0.00,94.02,76.43,46.00,38.91,...,88.79,NaN,NaN,98.19,68.24,NaN,87.62,0.0,5.56,100.00
4,90.87,90.95,89.31,93.20,87.07,0.00,96.36,65.96,8.51,90.06,...,85.67,NaN,NaN,98.51,0.00,NaN,30.57,0.0,50.00,99.18


In [42]:
df_perpixel_mean = df_perpixel.mean().round(2)
df_perpixel_mean

OA             87.77
wF1            87.54
wF1_Urban      84.01
wF1_Natural    92.89
F1_Class_1     86.74
F1_Class_2      7.55
F1_Class_3     95.07
F1_Class_4     62.66
F1_Class_5     14.39
F1_Class_6     73.92
F1_Class_7       NaN
F1_Class_8     87.48
F1_Class_9     57.47
F1_Class_10     9.52
F1_Class_11    95.60
F1_Class_12    48.67
F1_Class_13      NaN
F1_Class_14    53.72
F1_Class_15     0.00
F1_Class_16    22.13
F1_Class_17    98.35
dtype: float64

In [43]:
# export all results to csv
df_perpixel.to_csv(r"results\s2\saopaulo_S2_results.csv")

In [44]:
# export confusion matrices
with open(r"results\s2\saopaulo_S2_confusion_matrices.pkl", "wb") as f:
    pickle.dump(confusion_matrices, f)